In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
%reload_ext autoreload
%autoreload 2
from HelperFunctions import feature_standardize, label_encode_column, columns_of_type

from sklearn.model_selection import GridSearchCV
from sklearn.metrics.scorer import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
reg = RandomForestRegressor()
clf = RandomForestClassifier()

import pickle

In [2]:
total_df=pd.read_csv('Sampled_Dataset_Balanced.csv')
# total_df=pd.read_csv('down_sampled_df_v2.csv')
temp_pre_df=pd.read_csv('pre_downsample_df.csv')
pre_df=temp_pre_df.sample(100000)

In [3]:
# Defining the time windows:
# timeWindows=[('2007-07','2008-06'),
#              ('2008-07','2008-09'),('2008-10','2008-12'),
#              ('2009-01','2009-03'),('2009-04','2009-06'),
#              ('2009-07','2009-09'),('2009-10','2009-12'),
#              ('2010-01','2010-03'),('2010-04','2010-06'),
#              ('2010-07','2010-09'),('2010-10','2010-12'),
#              ('2011-01','2011-03'),('2011-04','2011-06'),
#              ('2011-07','2011-09'),('2011-10','2011-12'),
#              ('2012-01','2012-03'),('2012-04','2012-06'),
#              ('2012-07','2012-09'),('2012-10','2012-12'),
#              ('2013-01','2013-03'),('2013-04','2013-06'),
#              ('2013-07','2013-09'),('2013-10','2013-12'),
#              ('2014-01','2014-03'),('2014-04','2014-06'),
#              ('2014-07','2014-09'),('2014-10','2014-12'),
#              ('2015-01','2015-03'),('2015-04','2015-06'),
#              ('2015-07','2015-09'),('2015-10','2015-12'),
#              ('2016-01','2016-03'),('2016-04','2016-06'),
#              ('2016-07','2016-09'),('2016-10','2016-12'),
#              ('2017-01','2017-03'),('2017-04','2017-06'),
#              ('2017-07','2017-09'),('2017-10','2017-12'),
#              ('2018-01','2018-03'),('2018-04','2018-06'),
#              ('2018-07','2018-09'),('2018-10','2018-12'),
#             ]
timeWindows=[('2007-07','2008-06'),
             ('2008-07','2008-12'),
             ('2009-01','2009-06'),
             ('2009-07','2009-12'),
             ('2010-01','2010-06'),
             ('2010-07','2010-12'),
             ('2011-01','2011-06'),
             ('2011-07','2011-12'),
             ('2012-01','2012-06'),
             ('2012-07','2012-12'),
             ('2013-01','2013-06'),
             ('2013-07','2013-12'),
             ('2014-01','2014-06'),
             ('2014-07','2014-12'),
             ('2015-01','2015-06'),
             ('2015-07','2015-12'),
             ('2016-01','2016-06'),
             ('2016-07','2016-12'),
             ('2017-01','2017-06'),
             ('2017-07','2017-12'),
             ('2018-01','2018-06'),
             ('2018-07','2018-12'),
            ]
total_df.issue_d=pd.to_datetime(total_df.issue_d)
pre_df.issue_d=pd.to_datetime(pre_df.issue_d)

In [4]:
file_clf = open("pickle_model_clf.pkl",'wb')
file_time_grade_default_rate = open("pickle_time_grade_default_rate.pkl",'wb')
file_reg_def = open("pickle_model_reg_def.pkl",'wb')
file_reg_paid = open("pickle_model_reg_paid.pkl",'wb')

In [5]:
grade_weight_dict={'A':1,
            'B':1,
            'C':1,
            'D':1,
            'E':1,
            'F':1,
            'G':1}

In [6]:
# default rate per time frame (to address downsampling)

In [7]:
def grade_default_rate(unsampled_df): 
    # Finding the Default rate of each grade (of a given time)
    grade_rate=unsampled_df.groupby('grade')['loan_status'].apply(lambda x:(x=='Default').sum()/x.count())
    grade_rate_dict=grade_rate.to_dict()
    return grade_rate_dict
       
# def adjust_prob(grade_rate_dict,df,pre_adjust_prob)
#     pre_adjust_prob_df=pd.DataFrame(pre_adjust_prob[:,0],columns=['pre_adjust_prob'])
#     df['grade_default_rate']=df['grade'].map(grade_rate_dict)
#     df.reset_index(drop=True, inplace=True)
#     pre_adjust_prob_df.reset_index(drop=True, inplace=True)
#     pre_adjust_df=pd.concat([df,pre_adjust_prob_df],axis=1)
#     # Adjusting the default_probability to the true probability (accounting for down/up sampling)    
#     sampled_frac=0.5
#     real_prob=[]
#     for row in pre_adjust_df.loc[:,['grade_default_rate','pre_adjust_prob']].iterrows():
#         beta=sampled_frac/(1-row[1]['grade_default_rate'])
#         real_prob.append(beta*row[1]['pre_adjust_prob']/((beta-1)*row[1]['pre_adjust_prob']+1))
#     a=pd.DataFrame(real_prob,columns=['actual_prob'])
#     b=pd.DataFrame(pre_adjust_prob[:,0],columns=['pre_adjust_prob'])
#     return pd.concat([a,b],axis=1)

In [8]:
for timeIncrement in timeWindows:
    # Slicing the dataframe into incremental ones based on time
    print(timeIncrement)
    df=total_df.loc[(total_df.issue_d>=timeIncrement[0]) & (total_df.issue_d<=timeIncrement[1]),:]
    pre_sample_df=pre_df.loc[(pre_df.issue_d>=timeIncrement[0]) & (pre_df.issue_d<=timeIncrement[1]),:]

    df.loc[:,'weight']=df.loc[:,'grade'].map(grade_weight_dict)
    grade_weight_array=df.loc[:,'weight'].values
    index = df.index
    grade_weight_frame = pd.DataFrame(grade_weight_array, index=index)

#     drop_list=['id','sub_grade','issue_d','zip_code','loan_duration','loan_status','return_rate','weight','RANDOM']
    drop_list=['sub_grade','issue_d','duration','loan_status','return_rate','weight','total_payout']

    x_train_clf=df.drop(drop_list,axis=1)
    x_train_reg_paid=df.loc[df.loc[:,'loan_status']=='Fully Paid',:].drop(drop_list,axis=1)
    x_train_reg_default=df.loc[df.loc[:,'loan_status']=='Default',:].drop(drop_list,axis=1)
    y_train_clf=df.loc[:,'loan_status']
    y_train_reg_paid=df.loc[df.loc[:,'loan_status']=='Fully Paid',:].loc[:,'return_rate']
    y_train_reg_default=df.loc[df.loc[:,'loan_status']=='Default',:].loc[:,'return_rate']

    cat_list=columns_of_type(x_train_clf,'string')

    x_train_clf.reset_index(drop=True,inplace=True)
    x_train_clf=label_encode_column(x_train_clf,cat_list)
    x_train_reg_paid.reset_index(drop=True,inplace=True)
    x_train_reg_paid=label_encode_column(x_train_reg_paid,cat_list)
    x_train_reg_default.reset_index(drop=True,inplace=True)
    x_train_reg_default=label_encode_column(x_train_reg_default,cat_list)

    grid_para_forest = {
    'n_estimators': [10,20,30],
    'max_depth':[3,4,5],
    'max_features':['auto'],
    'min_samples_leaf':[10,15,20],
    'min_samples_split':[10,15,20]
    }
    ######################################################################
    ########### Regression Score ######################
    def grade_weighted_r2(y_true,y_pred,sample_weight):        

            weighted_r2=r2_score(y_true,y_pred,sample_weight=sample_weight.loc[y_true.index.values].values.reshape(-1))
            return weighted_r2    

    score_params = {"sample_weight": grade_weight_frame}

    grade_weighted_scorer_r2 = make_scorer(score_func=grade_weighted_r2,
                                        greater_is_better=True,
                                        needs_proba=False,
                                        needs_threshold=False,
                                        **score_params)
    ########### Default Regression ####################
    grid_search_forest_reg_default = GridSearchCV(estimator = RandomForestRegressor(), param_grid = grid_para_forest,
                                          scoring=grade_weighted_scorer_r2,
                                          cv = 5, n_jobs = -1,iid=False)
    grid_search_forest_reg_default.fit(x_train_reg_default,y_train_reg_default)
    print('default reg score:',grid_search_forest_reg_default.score(x_train_reg_default,y_train_reg_default))
    ########### Fully-Paid Regression #################
    grid_search_forest_reg_paid = GridSearchCV(estimator = RandomForestRegressor(), param_grid = grid_para_forest,
                                          scoring=grade_weighted_scorer_r2,
                                          cv = 5, n_jobs = -1,iid=False)
    grid_search_forest_reg_paid.fit(x_train_reg_paid,y_train_reg_paid)
    print('fully paid reg score:',grid_search_forest_reg_paid.score(x_train_reg_paid,y_train_reg_paid))
    ######################################################################   
    ########### Classification Score ###############
    def grade_weighted_accuracy(y_true,y_pred,sample_weight):
            weighted_acc=accuracy_score(y_true,y_pred,
                                        sample_weight=sample_weight.loc[y_true.index.values].values.reshape(-1),
                                        normalize=True)
            return weighted_acc 

    score_params = {"sample_weight": grade_weight_frame}

    grade_weighted_scorer_acc = make_scorer(score_func=grade_weighted_accuracy,
                                        greater_is_better=True,
                                        needs_proba=False,
                                        needs_threshold=False,
                                        **score_params)
    ########### Classification #####################
    grid_search_forest_clf = GridSearchCV(estimator=RandomForestClassifier(),\
                                          param_grid=grid_para_forest,\
                                          n_jobs=-1,\
                                          scoring=grade_weighted_scorer_acc,cv=5,\
                                          return_train_score=False,iid=False)
    grid_search_forest_clf.fit(x_train_clf,y_train_clf)
    print('clf score:',grid_search_forest_clf.score(x_train_clf,y_train_clf))
    
    ########### Grade Default Rates for this time frame #####################
    time_grade_default_rate_dict=grade_default_rate(pre_sample_df)
    # To avoid dividing by zeros in future steps. dictionary values of 1 yield nan later on.
    for key, val in time_grade_default_rate_dict.items(): 
        if val == 1:
            print('grade weight adjust')
            time_grade_default_rate_dict[key]=0.95  
            print(key,':',time_grade_default_rate_dict[key])
    ##########################################################################
    # Saving to Pickle
    pickle.dump(grid_search_forest_clf, file_clf)
    pickle.dump(time_grade_default_rate_dict, file_time_grade_default_rate)
    pickle.dump(grid_search_forest_reg_default, file_reg_def)    
    pickle.dump(grid_search_forest_reg_paid, file_reg_paid)

('2007-07', '2008-06')


/home/auscheng/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/auscheng/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


default reg score: 0.12466186240379395
fully paid reg score: 0.5025067185350396
clf score: 0.7014925373134329
('2008-07', '2008-12')


/home/auscheng/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/auscheng/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


default reg score: -0.003643563902599789
fully paid reg score: 0.019068079566584295
clf score: 0.7592592592592593
('2009-01', '2009-06')


...

default reg score: 0.13518176822192363
fully paid reg score: 0.7313821503040295
clf score: 0.6400984966303784
('2018-01', '2018-06')


/home/auscheng/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/auscheng/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


default reg score: 0.19215801520373121
fully paid reg score: 0.6171515585044591
clf score: 0.6618236472945892
('2018-07', '2018-12')


/home/auscheng/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/auscheng/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


default reg score: 0.04899192860131374
fully paid reg score: 0.5719405321051072
clf score: 0.6965376782077393


In [18]:
file_clf.close()
file_reg_def.close()
file_reg_paid.close()
file_time_grade_default_rate.close()

In [10]:
# my_x,my_y=data_transform(df.sample(1000),drop_list,'loan_status',grade_weight_dict)
# my_x,my_y=data_transform(df,drop_list,'return_rate',grade_weight_dict)

In [11]:
# # Load from file
# with open('pickle_model_clf.pkl', 'rb') as file:
#     pickle_model = pickle.load(file)
    
# # Calculate the accuracy score and predict target values
# score = pickle_model.score(my_x, my_y)
# print("Test score: {0:.2f} %".format(100 * score))
# Ypredict = pickle_model.predict(my_x)

In [12]:
# with open('pickle_model_reg_def.pkl', 'rb') as file:
#     pickle_model = pickle.load(file)
#     pickle_model2 = pickle.load(file)
#     pickle_model3 = pickle.load(file)
#     pickle_model4 = pickle.load(file)

# # Calculate the accuracy score and predict target values
# score = pickle_model.score(my_x, my_y)
# print("Test score: {0:.2f} %".format(100 * score))
# Ypredict = pickle_model.predict(my_x)

In [13]:
# with open('pickle_model_reg_paid.pkl', 'rb') as file:
#     pickle_model = pickle.load(file)
#     pickle_model2 = pickle.load(file)
#     pickle_model3 = pickle.load(file)
#     pickle_model4 = pickle.load(file)

# # Calculate the accuracy score and predict target values
# score = pickle_model.score(my_x, my_y)
# print("Test score: {0:.2f} %".format(100 * score))
# Ypredict = pickle_model.predict(my_x)

In [14]:
# score = pickle_model.score(my_x, my_y)
# print("Test score: {0:.2f} %".format(100 * score))
# Ypredict = pickle_model2.predict(my_x)

In [15]:
# score = pickle_model.score(my_x, my_y)
# print("Test score: {0:.2f} %".format(100 * score))
# Ypredict = pickle_model3.predict(my_x)

In [16]:
# score = pickle_model.score(my_x, my_y)
# print("Test score: {0:.2f} %".format(100 * score))
# Ypredict = pickle_model4.predict(my_x)

In [17]:
# score = pickle_model.score(my_x, my_y)
# print("Test score: {0:.2f} %".format(100 * score))
# Ypredict = pickle_model5.predict(my_x)